# 1. Preparation

In [ ]:
# Imports
from ppadb.client import Client as AdbClient # https://pypi.org/project/pure-python-adb/
import random
import sys
import time

# 2. Android Device Setup

## 2.1. Connect Device to Computer

### 2.1.1. Functions

In [ ]:
# List Available Devices function
def get_available_devices(client:AdbClient) -> list:
    
    # Get devices list
    devices = client.devices()

    # Print devices' serials list
    print(f'Available Devices: {[device.serial for device in devices]}')
    
    # Return devices list
    return devices

In [ ]:
# Connect Device function
def device_connect(client:AdbClient, device_serial:str):
    
    # Get devices list
    devices = client.devices()
    
    # Check if empty devices list
    if len(devices) == 0:
        sys.exit('No available devices. Exiting program...')

    # Go through devices list
    for device in devices:
        
        # If desired device found:
        if device.serial == device_serial:
    
            # Show "connected" message
            print (f'Connected to {device.serial}')        

            # Return device address
            return device
        
    # If desired device not found:
    print(f'Device {device_serial} not found.')
    
    # Return nothing
    return None

### 2.1.2. Connecting

In [ ]:
# Create client object
client = AdbClient(
    host="127.0.0.1",   # Default: "127.0.0.1"
    port=5037           # Default: 5037
)

# Get available devices list
devices = get_available_devices(client)

# Connect to first available device
device = device_connect(client, devices[0].serial)

## 2.2. Get Device's Screen Resolution

### 2.2.1. Functions

In [ ]:
# Get Screen Resolution Function
def get_screen_res(device:AdbClient) -> tuple:
    
    # Get device's screen physical size data (as STR)
    scrn_data = device.shell('wm size')

    # Remove "Physical size: " substring from screen data
    scrn_data = scrn_data.replace('Physical size: ', '')

    # Get screen's width and height values (as STRs)
    scrn_width, scrn_height = scrn_data.split(sep='x')

    # Cast width and height values as INTs
    scrn_width = int(scrn_width)
    scrn_height = int(scrn_height)

    # Return screen width and screen height
    return scrn_width, scrn_height

### 2.2.2. Getting screen resolution

In [ ]:
# Get device's screen resolution
x_max, y_max = get_screen_res(device=device)

# 3. Posting Offers

## 3.1. Push offer image to device's internal storage

In [ ]:
# Push offer JPG file from computer to device
device.push("../resources/story_template_720x1280_final.jpg", "/storage/emulated/0/DCIM/temp/offer.jpg")

# Make device "recognize" JPG file as media
device.shell('am broadcast -a android.intent.action.MEDIA_SCANNER_SCAN_FILE -d file:///storage/emulated/0/DCIM/temp/offer.jpg')

## 3.2. Post Offer as Instagram Story

### 3.2.1. Functions

In [ ]:
# Wait function
def wait(tmin=0.0, dt=0.0) -> None:
    
    # Get random float within [0, 1]
    x = round(random.uniform(0, 1), 6)

    # Calculate t
    t = tmin + (x * dt)

    # Sleep for t seconds
    time.sleep(t)

    # Debug
    print(f'Waited for {t:.6f} seconds.')

    # Return None
    return None

In [ ]:
# Input Touchscreen Tap function
def input_touchscreen_tap(
    device: AdbClient,
    x0: float,
    x1: float,
    xmax: int,
    y0: float,
    y1: float,
    ymax: int
) -> None:

    # Get random x within [x0*xmax , x1*xmax]
    x = round(random.SystemRandom().uniform(x0,x1)*xmax)

    # Get random y within [y0*ymax , y1*ymax]
    y = round(random.SystemRandom().uniform(y0,y1)*ymax)

    # Input touchscreen tap
    device.shell(f'input touchscreen tap {x} {y}')

    # Debug
    print(f'Tapped on (x,y)=({x}, {y})')

    # Return nothing
    return None


In [ ]:
# Drag Link Sticker function
def input_touchscreen_swipe(
    device: AdbClient,
    x00: float,
    x01: float,
    xmax: int,
    dx: int,
    y00: float,
    y01: float,
    ymax: int,
    dy: float,
    delay: int
) -> None:
    
    # Get random x0 within [x00*xmax , x01*xmax]
    x0 = round(random.SystemRandom().uniform(x00,x01)*xmax)

    # Get random y0 within [y0*ymax , y1*ymax]
    y0 = round(random.SystemRandom().uniform(y00,y01)*ymax)

    # Calculate x1, y1
    x1 = x0 + dx
    y1 = y0 + dy

    # Input drag-and-drop command into adb shell
    device.shell(f'input draganddrop {x0} {y0} {x1} {y1} {delay}')

    # Debug
    print(f'Drag-and-drop from (x,y)=({x0}, {y0}) to (x,y)=({x1}, {y1})')

    # Return nothing
    return None

### 3.2.2. Posting

In [ ]:
# Launch (re-launch) Instagram app on device
device.shell('am force-stop com.instagram.android')
device.shell('monkey -p com.instagram.android 1')

# Wait
wait(tmin=2, dt=0)

In [ ]:
# Click on "New post" button
input_touchscreen_tap(
    device=device,
    x0=0.416667,    # 450px/1080px
    x1=0.583330,    # 630px/1080px
    xmax=x_max,
    y0=0.895834,    # 2150px/2400px
    y1=0.9375,      # 2250px/2400px
    ymax=y_max
)

# Wait a little
wait(tmin=1, dt=0)

In [ ]:
# Click on "Story" option
input_touchscreen_tap(
    device=device,
    x0=0.574075,    # 620px/1080px
    x1=0.722222,    # 780px/1080px
    xmax=x_max,
    y0=0.858334,    # 2060px/2400px
    y1=0.889583,    # 2135px/2400px
    ymax=y_max
)

# Wait a little
wait(tmin=1, dt=0)

In [ ]:
# Click on "Open Phone Gallery" button
input_touchscreen_tap(
    device=device,
    x0=0.0092593,   # 10px/1080px
    x1=0.157407,    # 170px/1080px
    xmax=x_max,
    y0=0.845834,    # 2030px/2400px
    y1=0.904166,    # 2170px/2400px
    ymax=y_max
)

# Wait a little
wait(tmin=1.2, dt=0)

In [ ]:
# Select first gallery image
input_touchscreen_tap(
    device=device,
    x0=0.0092593,   # 10px/1080px
    x1=0.3240741,   # 350px/1080px
    xmax=x_max,
    y0=0.2875,      # 690px/2400px
    y1=0.533333,    # 1280px/2400px
    ymax=y_max
)

# Wait a little
wait(tmin=1.0, dt=0)

In [ ]:
# Click on "Add Sticker" button
input_touchscreen_tap(
    device=device,
    x0=0.5092593,   # 550px/1080px
    x1=0.5925925,   # 640px/1080px
    xmax=x_max,
    y0=0.0625,      # 150px/2400px
    y1=0.1,         # 240px/2400px
    ymax=y_max
)

# Wait a little
wait(tmin=0.8, dt=0)

In [ ]:
# Click on "Search" field
input_touchscreen_tap(
    device=device,
    x0=0.055556,    # 60px/1080px
    x1=0.925925,    # 1000px/1080px
    xmax=x_max,
    y0=0.229167,    # 550px/2400px
    y1=0.254166,    # 610px/2400px
    ymax=y_max
)

# Wait a little
wait(tmin=1, dt=0)

In [ ]:
# Type "link" on search field
device.shell('input text "link"')

# Wait a little
wait(tmin=0.1, dt=0)

In [ ]:
# Click on "Link" sticker
input_touchscreen_tap(
    device=device,
    x0=0.0462963,   # 50px/1080px
    x1=0.212962,    # 230px/1080px
    xmax=x_max,
    y0=0.254167,    # 610px/2400px
    y1=0.325,       # 780px/2400px
    ymax=y_max
)

# Wait a little
wait(tmin=1, dt=0)

In [ ]:
# Input offer URL
offer_url = 'https://redir.lomadee.com/v2/27067890937'
device.shell(f'input text "{offer_url}"')

# Wait a little
wait(tmin=0.1, dt=0)

In [ ]:
# Click on "Customize sticker text"
input_touchscreen_tap(
    device=device,
    x0=0.055556,    # 60px/1080px
    x1=0.925925,    # 1000px/1080px
    xmax=x_max,
    y0=0.304167,    # 730px/2400px
    y1=0.316666,    # 760px/2400px
    ymax=y_max
)

# Wait a little
wait(tmin=0.5, dt=0)

In [ ]:
# Input sticker text
sticker_text = 'ver oferta'
device.shell(f'input text "{sticker_text}"')

# Wait a few seconds
wait(tmin=0.1, dt=0)

In [ ]:
# Click on "Done"
input_touchscreen_tap(
    device=device,
    x0=0.851852,    # 920px/1080px
    x1=0.981481,    # 1060px/1080px
    xmax=x_max,
    y0=0.1375,      # 330px/2400px
    y1=0.1875,      # 450px/2400px
    ymax=y_max
)

# Wait a little
wait(tmin=0.5, dt=0)

In [ ]:
# Change link sticker color (click it 3x)
for _ in range(3):
    input_touchscreen_tap(
        device=device,
        x0=0.444445,    # 480px/1080px
        x1=0.555555,    # 600px/1080px
        xmax=x_max,
        y0=0.420834,    # 1010px/2400px
        y1=0.45,        # 1080px/2400px
        ymax=y_max
    )

    # Wait a little
    wait(tmin=0.3, dt=0)

In [ ]:
# Drag link sticker to desired position
input_touchscreen_swipe(
    device=device,
    x00=0.601851852,    # 650px/1080px
    x01=0.680555555,    # 735px/1080px
    xmax=x_max,
    dx=0,
    y00=0.420833334,    # 1010px/2400px
    y01=0.458333333,    # 1100px/2400px
    ymax=y_max,
    dy=1060,
    delay=1500
)

# Wait a little
wait(tmin=0.5, dt=0)

In [ ]:
# # Click on "Your story" (and post offer)
# input_touchscreen_tap(
#     device=device,
#     x0=0.0694445,   # 75px/1080px
#     x1=0.2268518,   # 245px/1080px
#     xmax=x_max,
#     y0=0.864584,    # 2075px/2400px
#     y1=0.880833,    # 2115px/2400px
#     ymax=y_max
# )

# # Wait a little
# wait(tmin=5, dt=0)

## 3.3. Delete offer image from device's internal storage

In [ ]:
# Remove offer image from device's internal storage
device.shell('rm storage/emulated/0/DCIM/temp/offer.jpg')